# Tikz

In [1]:
#via https://github.com/robjstan/tikzmagic
import subprocess
import shlex
import shutil
import tempfile
from argparse import ArgumentParser
from base64 import b64encode
from os.path import isfile
from os import getcwd

from IPython.core.magic import register_line_cell_magic
from IPython.core.display import Image

LATEX_TEMPLATE = r'''
\documentclass[tikz,border={border}]{{standalone}}
\usepackage{{tikz,{latex_pkgs}}}
\usetikzlibrary{{{tikz_libs}}}
{latex_pre}
\begin{{document}}
{content}
\end{{document}}'''

@register_line_cell_magic
def tikz(line, cell=''):
    '''Format TikZ commands into a LaTeX document, compile, and convert.'''
    parser = ArgumentParser()
    parser.add_argument('-p', '--latex_packages', default='')
    parser.add_argument('-x', '--latex_preamble', default='')
    parser.add_argument('-l', '--tikz_libraries', default='')
    parser.add_argument('-i', '--input_file', default=None)
    parser.add_argument('-e', '--export_file', default=None)
    parser.add_argument('-s', '--scale', default=1, type=float)
    parser.add_argument('-b', '--border', default=4)
    parser.add_argument('--wrap', dest='wrap_env', action='store_true')
    parser.add_argument('--no-wrap', dest='wrap_env', action='store_false')
    parser.set_defaults(wrap_env=True)
    parser.add_argument('--debug', dest='debug_mode', action='store_true')
    parser.set_defaults(debug_mode=False)
    args = parser.parse_args(shlex.split(line))

    # prepare latex from template
    if args.input_file:
        # add content from input_file before rest of cell
        cell += r'\input{{{cwd}/{file}}}'.format(cwd=getcwd(),
                                                 file=args.input_file)

    if args.wrap_env:
        cell = r'\begin{tikzpicture}' + cell + r'\end{tikzpicture}'

    latex = LATEX_TEMPLATE.format(content=cell,
                                  border=args.border,
                                  latex_pre=args.latex_preamble,
                                  latex_pkgs=args.latex_packages,
                                  tikz_libs=args.tikz_libraries)

    if args.debug_mode:
        print(latex)

    # add current working directory to any export_file path
    if args.export_file:
        args.export_file = getcwd() + '/' + args.export_file

    # compile and convert, returning Image data
    return latex2image(latex, int(args.scale*300), args.export_file)

def latex2image(latex, density, export_file=None):
    '''Compile LaTeX to PDF, and convert to PNG.'''
    try:
        # make a temp directory, and name temp files
        temp_dir = tempfile.mkdtemp()
        temp_tex = temp_dir + '/tikzfile.tex'
        temp_pdf = temp_dir + '/tikzfile.pdf'
        temp_png = temp_dir + '/tikzfile.png'

        open(temp_tex, 'w').write(latex)
        # run LaTeX to generate a PDF
        sh_latex(in_file=temp_tex, out_dir=temp_dir)

        if not isfile(temp_pdf):
            raise Exception('pdflatex did not produce a PDF file.')

        if export_file:
            shutil.copyfile(temp_pdf, export_file)

         # convert PDF to PNG
        sh_convert(in_file=temp_pdf, out_file=temp_png, density=density)

        return Image(data=b64encode(open(temp_png, "rb").read()))
    finally:
        # remove temp directory
        shutil.rmtree(temp_dir)

# functions to run command line scripts
def sh_latex(in_file, out_dir):
    '''Compile XeLaTeX to generate a PDF.'''
    subprocess.call(['xelatex', '-output-directory', out_dir, in_file])

def sh_convert(in_file, out_file, density=96):
    '''Use ImageMagick to convert PDF to PNG.'''
    subprocess.call(['convert', '-density', str(density), in_file, out_file])

def load_ipython_extension(ipython):
    '''Load iPython extension. Empty as we don't need to do anything.'''
    pass

In [ ]:
#ip = get_ipython()
#ip.register_magics(AsymptoteMagic)

In [2]:
%%tikz
\tikzset{every node/.style={font=\sffamily,white}}

\node[fill=red] at (0,0) (a) {This};
\node[fill=blue] at (2,0) (b) {That};
\draw[->] (a) -- (b);

FileNotFoundError: [Errno 2] No such file or directory: 'xelatex'

### via https://tex.stackexchange.com/questions/158668/nice-scientific-pictures-show-off

In [ ]:
%%tikz
\documentclass{standalone}
\usepackage{pgfplots}    % plot stuff
\pgfplotsset{compat=1.6} % avoid warnings

\begin{document}
\begin{tikzpicture}
\begin{axis}[
  axis equal image,
  xmin=13,xmax=35,
  ymin=0,ymax=3,
  width=7in,
  xlabel={$x/D$ (-)},
  ylabel={$y/D$ (-)},
]
  \foreach \num in {1,2,...,18} {
    \addplot[black] file {time43.39stream\num.dat};                          
  }
  \draw[fill=black] (axis cs:15,1) rectangle (axis cs:16,2);
\end{axis}
\end{tikzpicture}
\end{document}


In [ ]:
%%tikz
\documentclass{standalone}
\usepackage{tikz}

% Vector Styles
\tikzset{
  load/.style   = {ultra thick,-latex},
  stress/.style = {-latex},
  dim/.style    = {latex-latex},
  axis/.style   = {-latex,black!55},
}

% Drawing View
\tikzset{dimetric2/.style={
  x={(0.935cm,-0.118cm)},
  y={(0.354cm, 0.312cm)},
  z={(0.000cm, 0.943cm)},
}}

\begin{document}
  \begin{tikzpicture}
    \node (origin) at (0,0) {}; % shift relative baseline
    \coordinate (O) at (2,3);
    \draw[fill=gray!10] (O) circle (1);
    \draw[fill=white] (O) circle (0.75) node[below,yshift=-1.125cm] {Signpost Cross Section};
    \draw[dim] (O) ++(-0.75,0) -- ++(1.5,0) node[midway,above] {$d_i$};
    \draw[dim] (O) ++(-1,1.25) -- ++(2,0) node[midway,above] {$d_o$}; 
    \foreach \x in {-1,1} {
      \draw (O) ++(\x,0.25) -- ++(0,1.25);
    }
  \end{tikzpicture}
  \begin{tikzpicture}[dimetric2]
        \coordinate (O) at (0,0,0);
        \draw[axis] (O) -- ++(6,0,0) node[right] {$x$};
        \draw[axis] (O) -- ++(0,6,0) node[above right] {$y$};
        \draw[axis] (O) -- ++(0,0,6) node[above] {$z$};
        \draw[fill=gray!50] (0,0,-0.5) circle (0.5); 
        \fill[fill=gray!50] (-0.46,-0.2,-0.5) -- (0.46,0.2,-0.5) -- (0.46,0.2,0) -- (-0.46,-0.2,0) -- cycle;
        \draw[fill=gray!20] (O) circle (0.5);
    \draw (0.46,0.2,-0.5) -- ++(0,0,0.5) node[below right,pos=0.0] {Fixed Support};
    \draw (-0.46,-0.2,-0.5) -- ++(0,0,0.5);
    \draw[fill=gray!10] (O) circle (0.2);
    \fill[fill=gray!10] (-0.175,-0.1,0) -- (0.175,0.1,0) -- ++(0,0,4) -- (-0.175,-0.1,4) -- cycle;
    \draw (-0.175,-0.1,0) -- ++(0,0,4);
    \draw (0.175,0.1,0) -- ++(0,0,4) node[right,midway] {Steel Post};
    \draw (4,0,3.95) -- ++(0,0,-1);
    \foreach \z in {0.5,0.75,...,5} {
      \draw[-latex] (-2*\z/5-0.2,0,\z) -- (-0.2,0,\z);
    }
    \draw[load] (0,0,4) -- ++(0,0,-1.25) node[right,xshift=0.1cm] {$F_{z1}$};
    \draw[fill=gray!20] (-0.25,-0.25,5) -- (4,-0.25,5) -- (4,+0.25,5) -- (-0.25,+0.25,5) -- cycle; 
    \draw[fill=gray!50] (+4.00,-0.25,4) -- (4,+0.25,4) -- (4,+0.25,5) -- (+4.00,-0.25,5) -- cycle; 
    \draw[fill=gray!10] (-0.25,-0.25,4) -- (4,-0.25,4) -- (4,-0.25,5) -- (-0.25,-0.25,5) -- cycle; 
    \draw (4.05,0,4) -- ++(1,0,0);
    \draw (4.05,0,5) -- ++(1,0,0);
    \draw[dim] (4.5,0,0) -- ++(0,0,4) node[midway,right] {$h_1$};
    \draw[dim] (4.5,0,4) -- ++(0,0,1) node[midway,right] {$h_2$};
    \draw[dim] (0,0,3.4) -- ++(4,0,0) node[midway,below] {$b_2$};
    \coordinate (P) at (2,-0.25,4.5);
    \draw (P) -- ++(0,0,0.25);
    \draw (P) -- ++(0.25,0,0);
    \draw[dim] (2.125,-0.25,4.5) -- ++(0,0,-0.5) node[midway,right] {$z_1$};
    \draw[dim] (2,-0.25,4.625) -- ++(-2,0,0) node[midway,below] {$x_1$};
    \draw[load] (2,-2.45,4.5) -- ++(0,2.2,0) node[pos=0.0,right,xshift=0.08cm] {$F_{y1}$};
    \draw[axis,dashed,-] (O) -- (0,0,5);
    \draw (0,0,5.5) -- ++(4,0,0) node[midway,above] {$w_{z}$};
    \foreach \x in {0,0.25,...,4} {
      \draw[-latex] (\x,0,5.5) -- ++(0,0,-0.5);
    }
    \draw (-0.2,0,0) -- ++(-2,0,5) node[above,xshift=0.5cm] {$w_{x}=\frac{z}{h_1+h_2} w_0$};
  \end{tikzpicture}
\end{document}

In [ ]:
\documentclass[border=0pt,pstricks]{standalone}
\usepackage{pst-coil,pstricks-add}
\usepackage[nomessages]{fp}

\FPset\CoilArm{0.25}
\FPset\CoilWidth{0.3}
\FPeval\CoilTurn{round(50/3:3)}
\FPeval\DeltaY{0.5}
\FPeval\Amp{1.5}
\FPeval\FPS{25}
\FPeval\Vx{2}% propagation speed
\FPeval\Period{1}% second

\psset
{
    coilarm=\CoilArm,
    coilwidth=\CoilWidth,
}


\newcommand\System[4][0]{% #1: frame, #2: x, #3: y, #4: label
    \uput[90](#2,4.25){#4}
    \FPeval\CoilHeight{round((4-(#3)-2*CoilArm)/(CoilWidth*CoilTurn):3)}
    \pszigzag[coilheight=\CoilHeight,linejoin=2](#2,4)(#2,#3)
    \ifnum#1=1
        \bgroup
            \psset{origin={#2,#3}}
            \psframe[dimen=inner,fillstyle=solid,fillcolor=black](-0.5,0)(0.5,-1)
            \psdot[linecolor=yellow](0,-0.5)
        \egroup
    \fi
}

\begin{document}
\FPeval\DeltaTime{round(1/\FPS:2)}
\FPeval\TotalFrame{round(\FPS*\Period:0)}
\multido{\n=0.00+\DeltaTime}{\TotalFrame}{%
\begin{pspicture*}[showgrid=false](-1.5,-2)(3.5,5)
    % Ceiling
    \psframe
    [
        fillstyle=vlines,
        hatchsep=2pt,
        hatchwidth=0.5\pslinewidth,
        hatchcolor=gray,
        hatchangle=45,
        %linestyle=none
    ](0,4)(2,4.25)
    % Spring without box
    \FPeval\Y{round(-DeltaY-Amp*cos(2*pi*\n/Period)+2:3)}
    \System[1]{1}{\Y}{A}
    \psplot[algebraic,linecolor=red,plotpoints=1000]
        {-1.5}{3.5}{-\DeltaY-\Amp*cos((2*\psPi/\Period)*((-\Vx*\n+x-1)/\Vx))+2-0.5}
\end{pspicture*}}

\end{document}